In [31]:
import numpy as np
import os, sys, math

In [39]:
# Parâmetros

t = 6
n = 2^t

q = next_prime(8*n) ; r = (q-1)//2
while r%n !=0:
    q = next_prime(q+1); r = (q-1)//2

print q,r,n

641 320 64


In [40]:
K.<z> = GF(q)[]
R.<z> = K.quotient(z^n+1)

In [44]:
# Gerador aproximadamente gaussiano

def D(alfa,size=None):
    assert alfa > 0 and 4*alfa < 1 , '"D": parâmetro "alfa" tem de ser positivo e limitado a 1/4'
    _rnd  = lambda s : int(round(math.fmod(s*alfa*q,1+r)))
    
    if size == None:
        return _rnd(np.random.standard_normal())
    else:
        assert size > 0, '"D": o número de elementos na amostra tem de ser positivo'
        return map(_rnd, np.random.standard_normal(size)) 
    
# Gerador binário

def binary(size=n):
    return list(np.random.choice([-1,1],size))
    
# Gerador RLWE
def RLWE(s,alfa=0.2):
    x = R.random_element(); e = R(D(alfa,n))
    return (x,x*s+e)

# auxs

def rnd(pol):
    return [k if k <= r else k - q for k in map(lift,pol.list())]


In [47]:
# Classe R-LWE crypt/decrypt

class RLWE_Crypt(object):
    def __init__(self,alfa=0.1):
        self.alfa = alfa 
        self.pk   = R.random_element()
        self.pub  = RLWE(self.pk,self.alfa)
    def encrypt(self,m):
        (x,y) = self.pub 
        (w,c1) = RLWE(x,self.alfa) 
        e = r*R(m)+ R(D(self.alfa,n))
        c2 = y*w + e
        return (c1,c2)
    def decrypt(self,crypto):
        c1,c2 = crypto
        return [round(c/r) for c in rnd(c2 - c1*self.pk)]    

In [48]:
mess = binary()

C = RLWE_Crypt()
crypto = C.encrypt(mess)
print m
print C.decrypt(crypto)

[1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, 1]
[0, -1, -1, 1, 1, 0, 1, 1, 1, -1, 1, 0, 1, 0, 0, 0, 1, 1, -1, 0, 0, 0, 0, -1, -1, 0, 1, 1, 0, 0, -1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, -1, 0, -1, 0, 1, 0, 1, 0, 1, -1, -1, -1, 0, -1, 1, -1, 1, 1]
